Each move, the crab does the following actions:

 -  The crab picks up the three cups that are immediately clockwise of the current cup. They are removed from the circle; cup spacing is adjusted as necessary to maintain the circle.
 -  The crab selects a destination cup: the cup with a label equal to the current cup's label minus one. If this would select one of the cups that was just picked up, the crab will keep subtracting one until it finds a cup that wasn't just picked up. If at any point in this process the value goes below the lowest value on any cup's label, it wraps around to the highest value on any cup's label instead.
 -  The crab places the cups it just picked up so that they are immediately clockwise of the destination cup. They keep the same order as when they were picked up.
 -  The crab selects a new current cup: the cup which is immediately clockwise of the current cup.


In [ ]:
class Cup:
    __slots__ = 'label', 'next_cup'
    DIRECTORY = {}
    
    def __init__(self, label, nextcup=None):
        self.label = label
        self.next_cup = nextcup
        Cup.save_cup(self)
        
    @classmethod
    def find_cup(cls, label):
        return Cup.DIRECTORY[label]

    @classmethod
    def save_cup(cls, cup):
        Cup.DIRECTORY[cup.label] = cup

class CupGame:
    __slots__ = 'size', 'current_cup'
    def __init__(self, config, size = None, debug = False):
        
        if size is None:
            size = len(config)
        self.size = size
            
        self.current_cup = Cup(config[0], None)
        
        prev_cup = self.current_cup
        for label in config[1:]:
            next_cup = Cup(label, None)
            prev_cup.next_cup = next_cup
            prev_cup = next_cup
            
        if size > len(config):
            max_cup = len(config)
            more_cups = size - len(config)
            for i in range(more_cups):
                next_cup = Cup(i+max_cup + 1, None)
                prev_cup.next_cup = next_cup
                prev_cup = next_cup
                
        # last cup in the cycle has to complete the circle
        next_cup.next_cup = self.current_cup
        debug and print(f"Initial config {list(self.cups_starting_at(self.current_cup.label))}")
        
    def play1(self, debug = False):
        
        debug and print(f"Current cup ({self.current_cup.label})")
        
        # pick up three cups immediately clockwise
        # of the current cup
        picked_up = self.current_cup.next_cup
        labels = [
            picked_up.label,
            picked_up.next_cup.label,
            picked_up.next_cup.next_cup.label
        ]
        debug and print(f"Pick up {labels}")
        
        # The cup circle is readjusted
        self.current_cup.next_cup = picked_up.next_cup.next_cup.next_cup
        debug and print(f"Closed circle after removing {list(self.cups_starting_at(self.current_cup.label))}")
        
        # Find the cup with a label equal to the current
        # cup's label minus 1. 
        destination_label = self.current_cup.label - 1
        if destination_label < 1:
            destination_label = self.size
        
        # If this would select one of the cups that was just picked up, 
        # will keep subtracting one until find a cup that wasn't just picked up. 
        # If at any point the value goes below the lowest value 
        # on any label, wrap around to the highest value 
        while destination_label in labels:
            destination_label -= 1
            if destination_label < 1:
                destination_label = self.size
                
        debug and print(f"Destination is {destination_label}")
                
        # Place the cups just picked up so they are immediately 
        # clockwise of destination cup, keeping same order as
        # when picked up
        destination_cup = Cup.find_cup(destination_label)
        picked_up.next_cup.next_cup.next_cup = destination_cup.next_cup
        destination_cup.next_cup = picked_up
        
        # Select new current cup, immediately clockwise of 
        # the current cup
        self.current_cup = self.current_cup.next_cup
        
        debug and print(f"Final configuration {list(self.cups_starting_at(self.current_cup.label))}")
        
    def nrounds(self, n):
        for _ in range(n):
            self.play1()
            
    def cups_starting_at(self, cupnum):
        stoploss = 0
        cup = Cup.find_cup(cupnum)
        while True:
            yield cup.label
            cup = cup.next_cup
            if cup.label == cupnum:
                break
            stoploss += 1
            if stoploss > 25:
                break
            
    def score(self):
        return "".join([str(i) for i in self.cups_starting_at(1)])[1:]

In [ ]:
test_input = [3, 8,  9,  1,  2,  5,  4,  6,  7]
g = CupGame(test_input)
g.nrounds(10)
g.score()

In [ ]:
my_input = [ 4,6,3,5,2,8,1,7,9 ]
g = CupGame(my_input)
g.nrounds(100)
g.score()

# Problem 2

Due to what you can only assume is a mistranslation (you're not exactly fluent in Crab), you are quite surprised when the crab starts arranging many cups in a circle on your raft - one million (1000000) in total.

Your labeling is still correct for the first few cups; after that, the remaining cups are just numbered in an increasing fashion starting from the number after the highest number in your list and proceeding one by one until one million is reached. (For example, if your labeling were 54321, the cups would be numbered 5, 4, 3, 2, 1, and then start counting up from 6 until one million is reached.) In this way, every number from one through one million is used exactly once.

After discovering where you made the mistake in translating Crab Numbers, you realize the small crab isn't going to do merely 100 moves; the crab is going to do ten million (10000000) moves!

The crab is going to hide your stars - one each - under the two cups that will end up immediately clockwise of cup 1. You can have them if you predict what the labels on those cups will be when the crab is finished.

In [ ]:
# Let's try 20 before going on to a million
g = CupGame(test_input, 20, debug=True)
g.play1(debug=True)
g.play1(debug=True)

In [ ]:
g = CupGame(my_input, 1_000_000)
g.nrounds(10_000_000)

In [ ]:
result = g.cups_starting_at(1)
_ = next(result) # skip the 1
r1 = next(result)
r2 = next(result)

In [ ]:
r1*r2